<a href="https://colab.research.google.com/github/coa-project/pycoa/blob/main/coabook/zenodoAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/coa-project/pycoa/zenodo-backup)

# Zenodo REST API 

### load libraries

In [1]:
import json
import requests
import pydoi

import sys
import subprocess
import importlib
coa_module_info = importlib.util.find_spec("coaenv")
if coa_module_info != None:
  import coaenv as pycoa
else:
  #dans le cas où on utilise colab de Google  
  subprocess.check_call(['pip', 'install', '--quiet', 'git+https://github.com/coa-project/pycoa.git@ReadingDataModification'])
  sys.path.insert(1, 'pycoa')
  !pip install pandas==1.4.2
  import coa.front as pycoa

Defaulting to user installation because normal site-packages is not writeable


In [2]:
pycoa.getrawdb?

Signature: pycoa.getrawdb(**kwargs)
Docstring: Return the main pandas i.e with all the which values loaded from the database selected
File:      ~/lpnhe/pycoa/coa/front.py
Type:      function

In [3]:
importlib.util.find_spec("coaenv")

### Zenodo Community parameters

In [4]:
# Set the community identifier
community_id = "pycoa"

# Get the user's access token. Necesary for private or restricted contect
access_token = "YOUR_ACCESS_TOKEN"

### Get JSON Metadata from Zenodo

In [5]:
# Make a GET request to the `/api/communities/{community_id}/records` endpoint
headers = {"Authorization": f"Bearer {access_token}"}

# For private or restricted content use
#response = requests.get(f"https://zenodo.org/api/communities/{community_id}/records", headers=headers)

# For Public Items in Communities
response = requests.get(f"https://zenodo.org/api/communities/{community_id}/records")

response.raise_for_status()

# Parse the JSON response
entries = json.loads(response.content)

In [6]:
entries

{'hits': {'hits': [{'created': '2023-11-08T14:42:08.544450+00:00',
    'modified': '2023-11-08T14:42:08.810738+00:00',
    'id': 10083685,
    'conceptrecid': '10083684',
    'doi': '10.5281/zenodo.10083685',
    'conceptdoi': '10.5281/zenodo.10083684',
    'doi_url': 'https://doi.org/10.5281/zenodo.10083685',
    'metadata': {'title': 'mpoxgh',
     'doi': '10.5281/zenodo.10083685',
     'publication_date': '2023-11-08',
     'description': '<p>Monkeypox data for World from 2022-05-01 to 2023-10-19, including total_cases, total_deaths</p>',
     'access_right': 'open',
     'creators': [{'name': 'Our World in Data', 'affiliation': None}],
     'contributors': [{'name': 'PyCoa',
       'affiliation': None,
       'type': 'DataCurator'}],
     'keywords': ['World',
      'Monkeypox',
      'public health',
      'disease surveillance',
      'epidemiology'],
     'dates': [{'type': 'collected'},
      {'type': 'other',
       'description': 'Temporal Coverage',
       'start': '2022-05-

### Play with the entries

In [7]:
# Print PyCoa DF links for every database
for hit in entries["hits"]["hits"]:
    print("Title: " + hit["title"])
    print(hit["metadata"]["description"].rstrip("</p>").lstrip("<p>"))
    #print("DOI: " + hit["doi_url"].split(".org/")[-1:][0])
    for file in hit["files"]:
        if file["key"] == hit["title"]+"-PyCoa-DF.csv":
            print(file["key"] + " [Pycoa file]")
            print("URL: " + file["links"]["self"])
        else:
            print(file["key"] + " [Raw file]")
            print("URL: " + file["links"]["self"])
    print("\n")

Title: mpoxgh
Monkeypox data for World from 2022-05-01 to 2023-10-19, including total_cases, total_deaths
mpoxgh_owid-monkeypox-data.csv [Raw file]
URL: https://zenodo.org/api/records/10083685/files/mpoxgh_owid-monkeypox-data.csv/content
mpoxgh-PyCoa-DF.csv [Pycoa file]
URL: https://zenodo.org/api/records/10083685/files/mpoxgh-PyCoa-DF.csv/content


Title: sciensano
COVID-19 data for Belgium from 2020-03-06 to 2023-06-27, including cur_confirmed, cur_reporting, tot_hosp, tot_icu
sciensano-PyCoa-DF.csv [Pycoa file]
URL: https://zenodo.org/api/records/10083608/files/sciensano-PyCoa-DF.csv/content
sciensano_COVID19BE_CASES_MUNI.csv [Raw file]
URL: https://zenodo.org/api/records/10083608/files/sciensano_COVID19BE_CASES_MUNI.csv/content
sciensano_COVID19BE_HOSP.csv [Raw file]
URL: https://zenodo.org/api/records/10083608/files/sciensano_COVID19BE_HOSP.csv/content


Title: spf
COVID-19 data for France from 2020-01-23 to 2023-06-30, including cur_cas, cur_hospitalises, cur_reanimation, cur_tx_

### Load PyCoa Pandas Database from Zenodo

In [8]:
url="https://zenodo.org/api/records/10082026/files/dgs-PyCoa-DF.csv/content"

database = pycoa.pd.read_csv(url)

#pycoa.loadDataBase(pycoa.pd.read_csv(url))

In [9]:
database

,date,where,tot_cases,codelocation
0,2020-03-12,Aveiro,30.0000,PT.AV
1,2020-03-13,Aveiro,NAN,PT.AV
2,2020-03-14,Aveiro,NAN,PT.AV
3,2020-03-15,Aveiro,NAN,PT.AV
4,2020-03-16,Aveiro,NAN,PT.AV
...,...,...,...,...
16825,2022-09-28,Évora,NAN,PT.EV
16826,2022-09-29,Évora,NAN,PT.EV
16827,2022-09-30,Évora,NAN,PT.EV
16828,2022-10-01,Évora,NAN,PT.EV


> The next steap could be building a function to load the pandas CSV into pycoa, comments???